# dataset/ingest

In [1]:
import os
from tqdm import *
from abcli import file
from abcli.modules import objects
from abcli import fullname
from abcli.plugins import cache
from roofAI.semseg.utils import visualize
from roofAI.dataset import RoofAIDataset, DatasetKind, MatrixKind
from roofAI.dataset.ingest.AIRS import ingest_AIRS
from IPython.display import Image
from abcli import string
from abcli.plugins import seed
import numpy as np

import abcli.logging
import logging

logger = logging.getLogger()

logger.info(f"{fullname()}, built on {string.pretty_date()}")

Found credentials in shared credentials file: ~/.aws/credentials
abcli-7.2383.1-2023-06-aws-batch-a, built on 21 October 2023, 12:44:24


In [2]:
cache_object_name = cache.read("roofAI_ingest_AIRS_cache")
ingest_object_name=objects.unique_object("roofAI-AIRS-ingest")

📂 roofAI-AIRS-ingest-2023-10-21-12-44-25-11766


In [3]:
cache_object_path = objects.object_path(cache_object_name)
ingest_object_path = objects.object_path(ingest_object_name)
logger.info(f"-> {ingest_object_path}")

-> /Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766


In [4]:
counts = {
    "test": 10,
    "train": 10,
    "val": 10
}

chip_width = 480
chip_height = 360
chip_overlap = 0.1

log=True

In [5]:
cache_dataset = RoofAIDataset(cache_object_path)
ingest_dataset = RoofAIDataset(ingest_object_path, kind=DatasetKind.CAMVID).create(log=log)

RoofAIDataset[DatasetKind.AIRS:AIRS](roofAI_ingest_AIRS_cache_2023-10-15-15-53-41-09195): 3 subset(s): 95 X test + 857 X train + 94 X val
RoofAIDataset[DatasetKind.CAMVID:AIRS](roofAI-AIRS-ingest-2023-10-21-12-44-25-11766): 3 subset(s): 0 X test + 0 X train + 0 X val
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/CamVid/test)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/CamVid/testannot)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/CamVid/train)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/CamVid/trainannot)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/CamVid/val)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/Ca

In [7]:
def slice_matrix(
    input_matrix: np.ndarray,
    kind: MatrixKind,
    chip_height: int,
    chip_width: int,
    chip_overlap: float,
    max_chip_count: int,
    path: str,
    prefix:str,
    log:bool=False,
) -> int:
    if log:
        logger.info("slice_matrix[{}]: {} -{}X{}x{}-@{:.0f}%-> {} - {}".format(
            string.pretty_shape_of_matrix(input_matrix),
            kind,
            max_chip_count,
            chip_height,
            chip_width,
            chip_overlap * 100,
            path,
            prefix,
        ))

    count = 0
    for y in trange(0,input_matrix.shape[0] - chip_height, int(chip_overlap * chip_height)):
        for x in trange(0,input_matrix.shape[1] - chip_width, int(chip_overlap * chip_width)):
            count += 1

            chip = input_matrix[
                y:y+chip_height,
                x:x+chip_width
                ]

            assert file.save_image(
                os.path.join(
                    path,
                    "{}-{:010d}-{:010d}.png".format(
                        prefix,
                        y,
                        x,
                    )
                ),
                chip,
                log=log,
            )

            if count >= max_chip_count:
                return count

    return count

In [8]:
for subset in tqdm(counts.keys()):
    for matrix_kind in list(MatrixKind):
        chip_count = counts[subset]
        for record_id in cache_dataset.subsets[subset]:
            input_matrix = cache_dataset.get_matrix(
                subset,
                record_id,
                matrix_kind,
                log=log,
            )

            chip_count -= slice_matrix(
                input_matrix,
                matrix_kind,
                chip_height,
                chip_width,
                chip_overlap,
                max_chip_count=chip_count,
                path=ingest_dataset.subset_path(subset,matrix_kind),
                prefix=record_id,
                log=log,
            )

            if chip_count <= 0:
                break

  0%|          | 0/3 [00:00<?, ?it/s]RoofAIDataset[roofAI_ingest_AIRS_cache_2023-10-15-15-53-41-09195].get_filename(test,christchurch_424,MatrixKind.IMAGE): /Users/kamangir/storage/abcli/roofAI_ingest_AIRS_cache_2023-10-15-15-53-41-09195/test/image/christchurch_424.tif
RoofAIDataset[roofAI_ingest_AIRS_cache_2023-10-15-15-53-41-09195].get_matrix(test,christchurch_424,MatrixKind.IMAGE): 10000x10000x3:uint8
slice_matrix[10000x10000x3:uint8]: MatrixKind.IMAGE -10X360x480-@10%-> /Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/CamVid/test - christchurch_424

-abcli.file: 360x480x3:uint8 -> /Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/CamVid/test/christchurch_424-0000000000-0000000000.png
-abcli.file: 360x480x3:uint8 -> /Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766/SegNet-Tutorial/CamVid/test/christchurch_424-0000000000-0000000048.png
-abcli.file: 360x480x3:uint8 -> /Users/kam

```python
assert ingest_AIRS(
    cache_path=cache_object_path,
    ingest_path=ingest_object_path,
    {
        "test": 10,
        "train": 10,
        "val": 10
    }
)
```

In [9]:
seed.log(ingest_object_path)

abcli select /Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-44-25-11766; open .


In [10]:
# END